In [1]:
#%%capture
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.3.1/spark-sql-kafka-0-10_2.12-3.3.1.jar"
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10_2.12/3.1.3/spark-streaming-kafka-0-10_2.12-3.1.3.jar"

In [1]:
#import os
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.3,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1 pyspark-shell'

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [25]:
spark = SparkSession.builder.appName("item_count").master("local").getOrCreate()
#TimestampType
customSchema = StructType() \
 .add("timestamp", TimestampType(), True) \
 .add("value", IntegerType(), True) \
 .add("user_id", StringType(), True)

In [26]:
lines = spark \
    .readStream \
    .option("header", True) \
    .schema(customSchema) \
    .csv("/home/jovyan/tmp/text")

In [27]:
lines.printSchema()
type(lines)

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: integer (nullable = true)
 |-- user_id: string (nullable = true)



pyspark.sql.dataframe.DataFrame

In [28]:
#lines = lines.withColumn("timestamp",to_timestamp(col("timestamp")))
#lines.printSchema()

In [8]:
#1. Tumbling with count
'''
result = lines \
        .groupBy(window(lines.timestamp, "5 minutes")) \
        .count().orderBy("window")
'''

In [11]:
#2. Tumbling with sum
'''
result = lines \
        .groupBy(window(lines.timestamp, "5 minutes")) \
        .sum().orderBy("window")
'''

In [13]:
#3. Sliding with count
'''
result = lines \
        .groupBy(window(lines.timestamp, "5 minutes", "3 minutes")) \
        .count().orderBy("window")
'''

In [15]:
#4. Sliding with sum
'''
result = lines \
        .groupBy(window(lines.timestamp, "5 minutes", "3 minutes")) \
        .sum().orderBy("window")
'''

In [20]:
#5 Session gap 2 mins with count
'''
result = lines \
    .groupBy("user_id", session_window(lines.timestamp, "5 minutes")) \
    .count().orderBy("session_window")
'''

In [32]:
#6 Session gap 2 mins with sum

result = lines \
    .groupBy("user_id", session_window(lines.timestamp, "10 minutes")) \
    .sum('value').orderBy("session_window")


In [30]:
result.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- session_window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- sum(value): long (nullable = true)



In [26]:
#result2 = lines.filter(lines.value>2)

In [27]:
#result2.printSchema()

In [ ]:
# Start running the query that prints the windowed word counts to the console
query = result \
    .writeStream \
    .outputMode('complete') \
    .format('console') \
    .option('truncate', 'false') \
    .start()

query.awaitTermination()